In [4]:
import pandas as pd
import seaborn as sns
import random
import matplotlib.pyplot as plt
from scipy.stats import shapiro
import requests
from scipy.stats import ttest_ind

In [10]:
pip install psycopg2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 1.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for psycopg2: filename=psycopg2-2.9.6-cp310-cp310-macosx_10_9_x86_64.whl size=133859 sha256=37588d21c64bd85559919ffd1c1c91c04eb150d18cbabeb57e32d82126e47255
  Stored in directory: /Users/ckz/Library/Caches/pip/wheels/2c/a9/a3/ce3040cb04477d73004ae61fa75abe9262d776c0892ddff41a
Successfully built psycopg2
Note: you may need to restart the kernel to use updated packages.


In [11]:
import psycopg2

In [5]:
# Since we will be extracting the data from the tables multiple times, instead of writing the same code multiple times, I created a function.
def query_table_to_csv(tablename, db_host, db_name, db_user, db_password, output_path, row_limit=None):
    # create a connection object
    conn = psycopg2.connect(
        host=db_host,
        database=db_name,
        user=db_user,
        password=db_password
    )

    # create a cursor object
    cursor = conn.cursor()

    
    # execute a query to select rows from the specified table
    query = f"SELECT * FROM {tablename} TABLESAMPLE SYSTEM (10)"
    if row_limit:
        query += f" LIMIT {row_limit}"
    cursor.execute(query)

    print(query)

    # fetch the results
    results = cursor.fetchall()

    # create a DataFrame from the results
    df = pd.DataFrame(results, columns=[desc[0] for desc in cursor.description])

    # close the cursor and connection
    cursor.close()
    conn.close()

    # save the DataFrame to a CSV file
    df.to_csv(output_path, index=False)

    # print a message to confirm success
    print(f"Results saved to {output_path}")


In [6]:
# set the database connection parameters
db_host = "lhl-data-bootcamp.crzjul5qln0e.ca-central-1.rds.amazonaws.com"
db_name = "mid_term_project"    
db_user = "lhl_student"
db_password = "lhl_student"

In [13]:
# Importing Flights data
query_table_to_csv(
    tablename="flights",
    db_host=db_host,
    db_name=db_name,
    db_user=db_user,
    db_password=db_password,
    output_path="/Users/ckz/Desktop/DATA_SCIENCE_BOOTCAMP/Midterm-project/flights.csv",
    row_limit=10000000
)




#/Users/ckz/Desktop/DATA_SCIENCE_BOOTCAMP/Midterm-project

SELECT * FROM flights TABLESAMPLE SYSTEM (10) LIMIT 10000000
Results saved to /Users/ckz/Desktop/DATA_SCIENCE_BOOTCAMP/Midterm-project/flights.csv


In [17]:
# Importing the saved CSV file into dataframe
flights_df = pd.read_csv("/Users/ckz/Desktop/DATA_SCIENCE_BOOTCAMP/Midterm-project/flights.csv")

# print the first few rows of the DataFrame
flights_df.head()

/var/folders/j1/j5f5974x3sj1crjx533p6k9h0000gn/T/ipykernel_82579/3260047064.py:2: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  flights_df = pd.read_csv("/Users/ckz/Desktop/DATA_SCIENCE_BOOTCAMP/Midterm-project/flights.csv")


,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2018-01-01,B6,B6,B6,880,B6,N794JB,880,12954,LGB,...,231.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-01,B6,B6,B6,885,B6,N337JB,885,12478,JFK,...,427.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-01,B6,B6,B6,886,B6,N337JB,886,14492,RDU,...,427.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-01,B6,B6,B6,889,B6,N318JB,889,11278,DCA,...,444.0,0.0,0.0,8.0,0.0,10.0,NaN,NaN,NaN,NaN
4,2018-01-01,B6,B6,B6,891,B6,N348JB,891,10721,BOS,...,1185.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
flights_df.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'dup', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'flights', 'distance',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime', 'no_name'],
      dtype='object')

In [30]:
flights_df.shape

(1591098, 42)

In [19]:
query_table_to_csv(
    tablename="passengers",
    db_host=db_host,
    db_name=db_name,
    db_user=db_user,
    db_password=db_password,
    output_path="/Users/ckz/Desktop/DATA_SCIENCE_BOOTCAMP/Midterm-project/passengers.csv",
    row_limit=100
)

SELECT * FROM passengers TABLESAMPLE SYSTEM (10) LIMIT 100
Results saved to /Users/ckz/Desktop/DATA_SCIENCE_BOOTCAMP/Midterm-project/passengers.csv


In [20]:
# Importing the saved CSV file into dataframe
passengers_df = pd.read_csv("/Users/ckz/Desktop/DATA_SCIENCE_BOOTCAMP/Midterm-project/passengers.csv")

# print the first few rows of the DataFrame
passengers_df.head()

,departures_scheduled,departures_performed,payload,seats,passengers,freight,mail,distance,ramp_to_ramp,air_time,...,dest_country,dest_country_name,aircraft_group,aircraft_type,aircraft_config,year,month,distance_group,class,data_source
0,0.0,1.0,264444.0,0.0,0.0,0.0,0.0,1938.0,0.0,0.0,...,CA,Canada,8,820,2,2019,7,4,P,IF
1,0.0,7.0,1841132.0,0.0,0.0,1391500.0,0.0,4414.0,0.0,0.0,...,US,United States,8,820,2,2019,7,9,P,IF
2,0.0,5.0,1307253.0,0.0,0.0,883080.0,0.0,3713.0,0.0,0.0,...,BE,Belgium,8,820,2,2019,7,8,P,IF
3,0.0,5.0,1307253.0,0.0,0.0,973940.0,0.0,3713.0,0.0,0.0,...,US,United States,8,820,2,2019,7,8,P,IF
4,0.0,12.0,2029775.0,0.0,0.0,1809775.0,0.0,4633.0,0.0,0.0,...,US,United States,8,817,2,2019,7,10,G,IF


In [22]:
query_table_to_csv(
    tablename="fuel_comsumption",
    db_host=db_host,
    db_name=db_name,
    db_user=db_user,
    db_password=db_password,
    output_path="/Users/ckz/Desktop/DATA_SCIENCE_BOOTCAMP/Midterm-project/fuel_consumption.csv",
    row_limit=10000000
)

SELECT * FROM fuel_comsumption TABLESAMPLE SYSTEM (10) LIMIT 10000000
Results saved to /Users/ckz/Desktop/DATA_SCIENCE_BOOTCAMP/Midterm-project/fuel_consumption.csv


In [24]:
query_table_to_csv(
    tablename="flights_test",
    db_host=db_host,
    db_name=db_name,
    db_user=db_user,
    db_password=db_password,
    output_path="/Users/ckz/Desktop/DATA_SCIENCE_BOOTCAMP/Midterm-project/flight_test.csv",
    row_limit=10000000
)

SELECT * FROM flights_test TABLESAMPLE SYSTEM (10) LIMIT 10000000
Results saved to /Users/ckz/Desktop/DATA_SCIENCE_BOOTCAMP/Midterm-project/flight_test.csv


In [33]:
# Importing the saved CSV file into dataframe
flight_test = pd.read_csv("/Users/ckz/Desktop/DATA_SCIENCE_BOOTCAMP/Midterm-project/flight_test.csv")

# print the first few rows of the DataFrame
flight_test.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
0,2020-01-01,WN,WN,WN,5024,WN,N256WN,5024,14107,PHX,"Phoenix, AZ",11433,DTW,"Detroit, MI",1120,1710,N,230,1,1671
1,2020-01-01,WN,WN,WN,1016,WN,N266WN,1016,14107,PHX,"Phoenix, AZ",11540,ELP,"El Paso, TX",2105,2210,N,65,1,347
2,2020-01-01,WN,WN,WN,5191,WN,N923WN,5191,14107,PHX,"Phoenix, AZ",11540,ELP,"El Paso, TX",1540,1655,N,75,1,347
3,2020-01-01,WN,WN,WN,3180,WN,N910WN,3180,14107,PHX,"Phoenix, AZ",11697,FLL,"Fort Lauderdale, FL",1140,1755,N,255,1,1972
4,2020-01-01,WN,WN,WN,6686,WN,N7832A,6686,14107,PHX,"Phoenix, AZ",11884,GEG,"Spokane, WA",1315,1500,N,165,1,1020


In [34]:
flight_test.shape

(67958, 20)

In [26]:
# Importing the saved CSV file into dataframe
fuel_consumption = pd.read_csv("/Users/ckz/Desktop/DATA_SCIENCE_BOOTCAMP/Midterm-project/fuel_consumption.csv")

# print the first few rows of the DataFrame
fuel_consumption.head()

,month,airline_id,unique_carrier,carrier,carrier_name,carrier_group_new,sdomt_gallons,satl_gallons,spac_gallons,slat_gallons,...,sdomt_cost,satl_cost,spac_cost,slat_cost,sint_cost,ts_cost,tdomt_cost,tint_cost,total_cost,year
0,10,20398.0,MQ,MQ,Envoy Air,3,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016
1,10,20405.0,N8,N8,National Air Cargo Group Inc d/ba National Air...,1,42978.0,0.0,0.0,0.0,...,100571.0,0.0,0.0,0.0,0.0,100571.0,120669.0,3188568.0,3309237.0,2016
2,10,20108.0,NC,NC,Northern Air Cargo Inc.,1,368895.0,0.0,0.0,0.0,...,645688.0,0.0,0.0,0.0,0.0,645688.0,935413.0,111437.0,1046850.0,2016
3,10,20416.0,NK,NK,Spirit Air Lines,3,22987667.0,0.0,0.0,1730254.0,...,38191955.0,0.0,0.0,2874663.0,2874663.0,41066618.0,38191955.0,2874663.0,41066618.0,2016
4,10,20397.0,OH,OH,PSA Airlines Inc.,2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016


In [27]:
fuel_consumption.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272 entries, 0 to 271
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   month              272 non-null    int64  
 1   airline_id         272 non-null    float64
 2   unique_carrier     272 non-null    object 
 3   carrier            272 non-null    object 
 4   carrier_name       272 non-null    object 
 5   carrier_group_new  272 non-null    int64  
 6   sdomt_gallons      272 non-null    float64
 7   satl_gallons       272 non-null    float64
 8   spac_gallons       272 non-null    float64
 9   slat_gallons       272 non-null    float64
 10  sint_gallons       272 non-null    float64
 11  ts_gallons         272 non-null    float64
 12  tdomt_gallons      272 non-null    float64
 13  tint_gallons       272 non-null    float64
 14  total_gallons      272 non-null    float64
 15  sdomt_cost         272 non-null    float64
 16  satl_cost          272 non

In [ ]:
fuel_consumption.shape

(272, 25)